data was scrapped on june 13 2020

In [1]:
#Import Libraries
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None


## Import and Cleaning

In [2]:
calendar = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/calendar.csv.gz');
listings1 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/listings.csv');
listings2 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/listings.csv.gz');
neighborhoods = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/neighbourhoods.csv');
reviews1 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/reviews.csv');
reviews2 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/reviews.csv.gz');

#Data Notes:

#locations are not exact (0-450 feet from actual address) due to how airbnb displays data. Therefore a heatmap will work well
#price is in dollars

c:\users\alex cerce\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (43,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Start by fixing the data types for each table

In [3]:
#Data cleaning functions used below

#I would prefer integers to have no trailing zeros

#This function will search a specifc dataframe column and return the number of non integer values and NAN values
#I am doing this because I want to keep the trailing zeros if there are values in the array 
#I wont be able to take away trailing zeros if there are any nan values
def find_noninteger(Dataframe,feature): #feature should be input as a string to be consistent with normal pandas format
    value=Dataframe[feature];
    float_count=0;na_count=0;index_nonint=[];index_na=[];
    for i in range(len(value)):
        if (value[i]).is_integer()==0 | pd.isna(value[i])==0:
            float_count +=1
            index_nonint.append(i)
        if pd.isna(value[i])==1:
            na_count +=1
            #index_na.append(i)
    if float_count==1 and na_count==1:
        print('There is',float_count,'non integer value and',na_count,'NAN value')
    elif float_count==1 and na_count!=1:
        print('There is',float_count,'non integer value and',na_count,'NAN values')
    elif float_count!=1 and na_count==1:
        print('There are',float_count,'non integer values and',na_count,'NAN value')
    elif float_count!=1 and na_count!=1:
        print('There are',float_count,'non integer values and',na_count,'NAN values')
    if index_nonint or index_na:
        print('The non integer values are located at:',index_nonint)

### Calendar Data Table
    The calendar table holds availibility data for every listing. The availibility is given from
    the date sampled up to one full year. Therefore, each listing will have 365 sample points.

In [4]:
calendar.head(n=5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,80843,2020-06-14,f,$39.00,$39.00,2.0,1125.0
1,186851,2020-06-15,f,$33.00,$33.00,62.0,300.0
2,186851,2020-06-16,f,$33.00,$33.00,62.0,300.0
3,186851,2020-06-17,f,$33.00,$33.00,62.0,300.0
4,186851,2020-06-18,f,$33.00,$33.00,62.0,300.0


In [5]:
#Missing Values
calendar_missingvalues_df=calendar[calendar.isna().any(axis=1)]    #dataframe containing only rows with any missing values
calendar_missingvalues_columns=calendar.columns[calendar.isna().any()].tolist()  #list of columns with missing values
print('These features have missing values:',calendar_missingvalues_columns)
print('\'minimum_nights\' has',calendar['minimum_nights'].isna().sum(),'missing values')
print('\'maximum_nights\' has',calendar['maximum_nights'].isna().sum(),'missing values')

# 'minimumn_nights' and 'maximum_nights':

#I have found that these 141 missing values are all from different listings, and they all
#occur on the sample date. This is just a data collection error
#I can pull the min and max nights values from other entries since they will be the same for the listing 

#pull the listing_id for the missing value, set missing values to be the same as the following row in full df
#use the sub table to reduce cpu load
for i in range(len(calendar_missingvalues_df)):
    missing_listing_id=calendar_missingvalues_df.iloc[i]['listing_id']
    missing_index=calendar_missingvalues_df.index[calendar_missingvalues_df['listing_id']==missing_listing_id].values[0]
    missing_listing_df=calendar.loc[calendar['listing_id']==missing_listing_id]
    
    calendar.at[missing_index,'minimum_nights']=missing_listing_df.iloc[50]['minimum_nights']  #choose arbitrary future date to set these values to
    calendar.at[missing_index,'maximum_nights']=missing_listing_df.iloc[50]['maximum_nights']
    
print('After filling missing values with the correct values:')
print('\'minimum_nights\' has',calendar['minimum_nights'].isna().sum(),'missing values')
print('\'maximum_nights\' has',calendar['maximum_nights'].isna().sum(),'missing values')


These features have missing values: ['minimum_nights', 'maximum_nights']
'minimum_nights' has 141 missing values
'maximum_nights' has 141 missing values
After filling missing values with the correct values:
'minimum_nights' has 0 missing values
'maximum_nights' has 0 missing values


In [ ]:
#check columns for decimal values. This will indicated whether or not I should remove trailing zeros (by converting to INT)
print('minimum nights:')
find_noninteger(calendar,'minimum_nights')
print('maximum nights:')
find_noninteger(calendar,'maximum_nights')
print('-'*50);print('Before:');print(calendar.dtypes)  #This indicates, that once I deal with missing values I can convert these cols to int


#Convert Data Types
#convert to datetime
calendar['date']=calendar['date'].astype('datetime64')
#convert to string dtype and capitalize T/F values
calendar['available']=calendar['available'].str.replace('f','F').str.replace('t','T').astype('str')
#remove dollar signs, remove comas, and convert to float
calendar['price']=calendar['price'].str.replace('$','').str.replace(',','').astype('float64') 
calendar['adjusted_price']=calendar['adjusted_price'].str.replace('$','').str.replace(',','').astype('float64')
#Convert to INT (no decimal place values and no more missing values)
calendar['minimum_nights']=calendar['minimum_nights'].astype(int)
calendar['maximum_nights']=calendar['maximum_nights'].astype(int)
print('-'*50);print('After:')
print(calendar.dtypes)


minimum nights:
There are 0 non integer values and 0 NAN values
maximum nights:


In [ ]:
calendar.head()

#number of observations in df
#number of unique listings

### Listings_1 Data Table

In [ ]:
listings1.head()

### Listings_2 Data Table

In [ ]:
listings2.head()

### Reviews_1 Data Table

In [ ]:
reviews1.head()

### Reviews_2 Data Table

In [ ]:
reviews2.head()

### Neighborhoods Data Table

In [ ]:
neighborhoods.head()
#Table which indicated the borough which each neighborhood belongs to

neighborhoods.columns = ['borough', 'neighborhood']   #rename columns 